<span id="menu"></span>
# Zipline Order函數介紹
Zipline Order是一個用於購買股票的函數，我們可以利用以下六種函數去塞選出我們指定要購買的股票價格以及數量，本篇將會介紹到order()的主要函數以及order_target()的使用辦法。
* [limit_price](#limit_price)
* [Stop_price](#Stop_price)
* [order_target](#order_target)

## Zipline有六種下單函數：
	order(): 購買指定股數
	order_value(): 購買指定價值
	order_percent(): 將整個投資組合 (portfolio value) 的一個比例拿去交易
	order_target(): 交易到該股票總股數達到指定數量為止
    order_target_value(): 交易到該股票價值達到指定價值為止
    order_target_percent(): 將股票在投資組合的比重調整到指定的比例

## 每個函數的parameters都有：
    asset: 股票或其他金融產品
    amount/value/percent: 數量、價值、比重，每個函數不同，但一律正 long 負 short
    limit_price: 限價，預設為None
    stop_price: 止損價，預設為None

## 規則：
    limit_price: 最高的買進價 (或最低的賣出價)
    stop_price: 當市場價超過stop price，則用市場價買入 (若是賣出，則是在股價跌低於stop price後用市場價賣出)
    Zipline 程式設計關係，今天下單才明天成交，limit/stop 是和第二日收盤價做比較
	若遇到股票分割，新的stop_price或limit_price都是原本數值乘以ratio後四捨五入到小數第二位


## 設定環境

In [1]:
import pandas as pd
import datetime
import tejapi
import time
import os
import warnings
warnings.filterwarnings('ignore')

# tej_key-------------------------------------------
tej_key ='your key'
tejapi.ApiConfig.api_key = tej_key
os.environ['TEJAPI_BASE'] = "https://api.tej.com.tw"
os.environ['TEJAPI_KEY'] = tej_key

# date----------------------------------------------
# set date
start='2018-07-24'
end='2018-08-14'
os.environ['mdate'] = '20180724 20180814'      # start+' '+end #'20221011 20221223'

tz = 'UTC'
start_dt, end_dt = pd.Timestamp(start, tz = tz), pd.Timestamp(end, tz = tz)
# calendar------------------------------------------
calendar_name='TEJ'  # US equities  XTAI

# bundle_name---------------------------------------
bundle_name = 'tquant'

from zipline.utils.calendar_utils import get_calendar
if get_calendar(calendar_name).is_session(start_dt)==False:
    start_dt=get_calendar(calendar_name).next_open(start_dt)

if get_calendar(calendar_name).is_session(end_dt)==False:
    end_dt=get_calendar(calendar_name).previous_close(pd.Timestamp(end_dt))

from zipline.api import    *

from zipline import run_algorithm
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume

from zipline.utils.run_algo import  (get_transaction_detail,
                                     get_record_vars)

from zipline.sources.TEJ_Api_Data import (get_Treasury_Return,
                                          get_Benchmark_Return,)



import numpy as np
import pandas as pd
from zipline.utils import run_algo
from zipline.data import bundles
coid='1101 1102 IR0001'


os.environ['ticker'] = coid       #'1101 1102'   #coid

!zipline ingest -b tquant

Merging daily equity files:


[2023-08-08 05:13:12.732288] INFO: zipline.data.bundles.core: Ingesting tquant.


<span id="limit_price"></span>
# Zipline Order 範例講解 -limit_price
## 從2018/07/24到2018/08/14，每天買一張1101，limit_price = 45
[Return to Menu](#menu)

####   limit_price = XX
       購買的價格必須低於XX才會買進

### 設定交易模型
#### initialize(context):
    在回測開始時被調用的函數，進行初始化設定
    - 這邊透過context參數來儲存和共享各種回測所需的變數和參數
* context.tickers = ['1101']
        定義一個股票代碼列表，這裡我們指定 1101 的股票資料。
* context.asset = [symbol(ticker) for ticker in context.tickers]
        將股票代碼輸入進 context 中。
* set_slippage(slippage.FixedSlippage(spread=0.00))
        設定滑價模型，這裡使用的是固定滑價模型，價差為 0。
* set_commission(commission.PerDollar(cost=commission_cost))
        設定交易費用模型，這裡設定一定比例的交易費用。
* set_benchmark(symbol('IR0001'))
        設定基準，將基準設為代碼為 'IR0001' 的資產，為市場報酬大盤。

In [2]:
def initialize(context):
    context.tickers = ['1101']
    context.asset = [symbol(ticker) for ticker in context.tickers]
    set_slippage(slippage.FixedSlippage(spread=0.00))
    set_commission(commission.PerDollar(cost=commission_cost))
    set_benchmark(symbol('IR0001'))

## 設定交易策略
#### handle_data(context, data):
     在每個交易日被調用的函數，用於處理交易資料
     -用於提供回測過程中的資料。通過data，我們可以獲取股票的歷史價格、成交量等資訊。
     -context被用來記錄是否已經下過單的狀態
     
   * for asset in context.asset: order(asset, 1000, limit_price = 45)：<br>
       對投資組合中的每個資產，下單購買 1000 單位，限價為 45。
       * record(close=data.current(context.asset, 'close'))<br>
           記錄每個資產的收盤價。

In [3]:
def handle_data(context, data):

    for asset in context.asset:
        order(asset, 1000, limit_price = 45)

    record(close=data.current(context.asset, 'close'))

### 設計回測結果(這邊不會作圖，因此不會用到)
#### analyze(context, perf):
    這個函數用於分析回測結果並繪製相關圖表
    -context用於存儲回測策略中的各種參數和變數
    -perf是一個DataFrame，其中包含回測的性能結果，例如投資組合價值、收益率等
 

In [4]:
def analyze(context, perf):

    pass

### 設定交易成本、回測
#### - commission_cost 交易成本設置
#### - capital_base 交易本金
#### - get_Treasury_Return 交易報酬率回測

In [5]:
commission_cost = 0.001425
capital_base = 1e6
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',
                                      term = '1y',
                                      symbol = '5844')

### 設置run_algorithm中的函數
run_algorithm是用於運行zipline回測的主要程式碼，通過設定回測的起始日期、結束日期、初始資本金和所使用的數據，並使用initialize、handle_data和 analyze函數來進行回測運算。<br>
#### positions, transactions, orders = get_transaction_detail(performance)
        使用get_transaction_detail將交易訊息分成以下三個函數:
            - positions 持倉狀態
            - transactions 交易紀錄
            - orders 訂單紀錄

In [7]:
closing_price = tejapi.get('TWN/APIPRCD',
                           coid=['1101'],
                           opts={'columns':['mdate','coid','close_d']},
                           mdate={'gte':start_dt,'lte':end_dt },
                           paginate=True)

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

positions, transactions, orders = get_transaction_detail(performance)

## 回測資料

### positions
    (查詢持倉狀態)
    這邊可以觀察到到7/26開始 amount(數量)的部分每一天都在增加1000單位

In [8]:
positions

,,,amount,cost_basis,last_sale_price
date,asset,symbol,,,
2018-07-26 00:00:00+08:00,Equity(0 [1101]),1101,2200,40.557713,40.50
2018-07-27 00:00:00+08:00,Equity(0 [1101]),1101,3200,40.495123,40.30
2018-07-30 00:00:00+08:00,Equity(0 [1101]),1101,4200,40.557712,40.70
2018-07-31 00:00:00+08:00,Equity(0 [1101]),1101,5200,40.336244,39.35
2018-08-01 00:00:00+08:00,Equity(0 [1101]),1101,6200,40.460800,41.05
2018-08-02 00:00:00+08:00,Equity(0 [1101]),1101,7200,40.488169,40.60
2018-08-03 00:00:00+08:00,Equity(0 [1101]),1101,8200,40.490544,40.45
2018-08-06 00:00:00+08:00,Equity(0 [1101]),1101,9200,40.481517,40.35
2018-08-07 00:00:00+08:00,Equity(0 [1101]),1101,10200,40.454625,40.15


### orders 訂單紀錄
    這邊我們可以觀察到在每日收盤價中7/24、以及7/25的部分收盤價都大於45，因此只有下單，沒有買入股票。
    同時因為7/26是除權日，拆股規則是1000股變1100股，他們新的limit變成45/1.1 = 40.9。
    
    因為7/26日收盤價40.5低於40.9，所以7/24、7/25兩單都在7/26成交(交易量都調整成了1100)
    可以發現在'date'為7/26的時候，'created'欄位7/24、7/25的單都被交易掉了，同時'limit_reached'開啟，交易金額限制在40.9以下
    
    補充：若原本limit設為43.5，其餘條件不變，那7/24、7/25下的那兩單，要一直等到7/31股價低於43.5/1.1=39.55後，才會各以當天收盤價39.35 * 1100股成交。

### closing_price 每日收盤價

In [9]:
orders

id  \
date                      asset            symbol                                     
2018-07-24 00:00:00+08:00 Equity(0 [1101]) 1101    f6018db8876a4e6e9792685428d62e7f   
2018-07-25 00:00:00+08:00 Equity(0 [1101]) 1101    8407eb1b97ca4a38a154759987c4caf4   
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101    f6018db8876a4e6e9792685428d62e7f   
                                           1101    8407eb1b97ca4a38a154759987c4caf4   
                                           1101    cd66d834b1864c1485e0e6706fca4f6c   
2018-07-27 00:00:00+08:00 Equity(0 [1101]) 1101    cd66d834b1864c1485e0e6706fca4f6c   
                                           1101    bf3b32bc3e7546d0b1de7222b03d7da4   
2018-07-30 00:00:00+08:00 Equity(0 [1101]) 1101    bf3b32bc3e7546d0b1de7222b03d7da4   
                                           1101    6e2c2b78f2514887b79fa043a417d11c   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101    6e2c2b78f2514887b79fa043a417d11c   
                                           1101    98cc5d933a9447918d8088d28e980416   
2018-08-01 00:00:00+08:00 Equity(0 [1101]) 1101    98cc5d933a9447918d8088d28e980416   
                                           1101    7f3a238f740b469e9b647fba8cefdab5   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101    7f3a238f740b469e9b647fba8cefdab5   
                                           1101    feaa9b52c1294095a4438a478733a8fe   
2018-08-03 00:00:00+08:00 Equity(0 [1101]) 1101    feaa9b52c1294095a4438a478733a8fe   
                                           1101    f5119a9417514f999f7ded6c2ed83280   
2018-08-06 00:00:00+08:00 Equity(0 [1101]) 1101    f5119a9417514f999f7ded6c2ed83280   
                                           1101    171c6a30fe994de18f6e0c75705e8d61   
2018-08-07 00:00:00+08:00 Equity(0 [1101]) 1101    171c6a30fe994de18f6e0c75705e8d61   
                                           1101    8f2f56cbe7b944558a9f74bced687ea2   
2018-08-08 00:00:00+08:00 Equity(0 [1101]) 1101    8f2f56cbe7b944558a9f74bced687ea2   
                                           1101    32bd68ef0c48413ea40453395c4a1cb5   
2018-08-09 00:00:00+08:00 Equity(0 [1101]) 1101    32bd68ef0c48413ea40453395c4a1cb5   
                                           1101    1fde9c798ee647cb97bc548f2f9bc975   
2018-08-10 00:00:00+08:00 Equity(0 [1101]) 1101    1fde9c798ee647cb97bc548f2f9bc975   
                                           1101    7595cacacbe84457bd82d767397473e0   
2018-08-13 00:00:00+08:00 Equity(0 [1101]) 1101    7595cacacbe84457bd82d767397473e0   
                                           1101    fdbdabe161c248ada7a1eabec6548c75   
2018-08-14 00:00:00+08:00 Equity(0 [1101]) 1101    fdbdabe161c248ada7a1eabec6548c75   
                                           1101    48bbca3bc36d4b988b90b6485cbe45f1   

                                                                         dt  \
date                      asset            symbol                             
2018-07-24 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-24 13:30:00+08:00   
2018-07-25 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-25 13:30:00+08:00   
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-26 13:30:00+08:00   
                                           1101   2018-07-26 13:30:00+08:00   
                                           1101   2018-07-26 13:30:00+08:00   
2018-07-27 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-27 13:30:00+08:00   
                                           1101   2018-07-27 13:30:00+08:00   
2018-07-30 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-30 13:30:00+08:00   
                                           1101   2018-07-30 13:30:00+08:00   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-31 13:30:00+08:00   
                                           1101   2018-07-31 13:30:00+08:00   
2018-08-01 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-01 13:30:00+08:00   
                                           1101   2018-08-01 13:30:00+08:00   
2018-08-02 00:00:00+08:0

In [10]:
closing_price

,mdate,coid,close_d
None,,,
0,2018-07-24,1101,45.50
1,2018-07-25,1101,45.10
2,2018-07-26,1101,40.50
3,2018-07-27,1101,40.30
4,2018-07-30,1101,40.70
5,2018-07-31,1101,39.35
6,2018-08-01,1101,41.05
7,2018-08-02,1101,40.60
8,2018-08-03,1101,40.45


### transactions 交易記錄


In [11]:
transactions

amount  \
date                      asset            symbol           
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101      1100   
                                           1101      1100   
2018-07-27 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-07-30 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-08-01 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-08-03 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-08-06 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-08-07 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-08-08 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-08-09 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-08-10 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-08-13 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-08-14 00:00:00+08:00 Equity(0 [1101]) 1101      1000   

                                                                         dt  \
date                      asset            symbol                             
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-26 13:30:00+08:00   
                                           1101   2018-07-26 13:30:00+08:00   
2018-07-27 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-27 13:30:00+08:00   
2018-07-30 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-30 13:30:00+08:00   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-31 13:30:00+08:00   
2018-08-01 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-01 13:30:00+08:00   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-02 13:30:00+08:00   
2018-08-03 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-03 13:30:00+08:00   
2018-08-06 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-06 13:30:00+08:00   
2018-08-07 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-07 13:30:00+08:00   
2018-08-08 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-08 13:30:00+08:00   
2018-08-09 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-09 13:30:00+08:00   
2018-08-10 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-10 13:30:00+08:00   
2018-08-13 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-13 13:30:00+08:00   
2018-08-14 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-14 13:30:00+08:00   

                                                   price  \
date                      asset            symbol          
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101    40.50   
                                           1101    40.50   
2018-07-27 00:00:00+08:00 Equity(0 [1101]) 1101    40.30   
2018-07-30 00:00:00+08:00 Equity(0 [1101]) 1101    40.70   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101    39.35   
2018-08-01 00:00:00+08:00 Equity(0 [1101]) 1101    41.05   
2018-08-02 00:00:00+08:00 Equity(0 [1101]) 1101    40.60   
2018-08-03 00:00:00+08:00 Equity(0 [1101]) 1101    40.45   
2018-08-06 00:00:00+08:00 Equity(0 [1101]) 1101    40.35   
2018-08-07 00:00:00+08:00 Equity(0 [1101]) 1101    40.15   
2018-08-08 00:00:00+08:00 Equity(0 [1101]) 1101    40.60   
2018-08-09 00:00:00+08:00 Equity(0 [1101]) 1101    40.50   
2018-08-10 00:00:00+08:00 Equity(0 [1101]) 1101    41.50   
2018-08-13 00:00:00+08:00 Equity(0 [1101]) 1101    42.15   
2018-08-14 00:00:00+08:00 Equity(0 [1101]) 1101    43.30   

                                                                           order_id  \
date                      asset            symbol                                     
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101    f6018db8876a4e6e9792685428d62e7f   
                                           1101    8407eb1b97ca4a38a154759987c4caf4   
2018-07-27 00:00:00+08:00 Equity(0 [1101]) 1101    cd66d834b1864c1485e0e6706fca4f6c   
2018-07-30 00:00:00+08:00 Equity(0 [1101]) 1101    bf3b32bc3e7546d0b1de7222b03d7da4   
2018-07-31 00:00:00+08:00 Equity(0 [1101]) 1101    6e2c2b78f2514887b79fa043a417d11c   
2018-08-01 00:00:00+08:00 Equity(0 [110

<span id="Stop_price"></span>
# Zipline Order 範例講解 - Stop_price
[Return to Menu](#menu)

## 從2018/07/24到2018/08/14，每天買一張1101，stop_price = 43
### stop_price = xx
    當價格大於xx時買入股票
### handle_data設置
    這邊的交易策略變成，每天買一張1101，購買的金額要大於43

In [12]:
def initialize(context):

    context.tickers = ['1101']
    context.asset = [symbol(ticker) for ticker in context.tickers]
    set_slippage(slippage.FixedSlippage(spread=0.00))
    set_commission(commission.PerDollar(cost=commission_cost))
    set_benchmark(symbol('IR0001'))

def handle_data(context, data):

    for asset in context.asset:
        order(asset, 1000, stop_price = 43)
    record(close=data.current(context.asset, 'close'))

def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e6
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',
                                      term = '1y',
                                      symbol = '5844')



In [14]:
closing_price = tejapi.get('TWN/APIPRCD',coid=['1101'], opts={'columns':['mdate','coid','close_d']}, mdate={'gte':start_dt,'lte':end_dt }, paginate=True)

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

positions, transactions, orders = get_transaction_detail(performance)

## 講解
    7/25(還沒分割)收盤價45.1已超過43所以7/24單子在7/25以45.1 * 1000股成交。7/26為除權日，7/25的單子處理邏輯是：購買量調整成1100，7/26收盤價40.5 > 43/1.1，所以以40.5 * 1100股成交。

    下面收盤價中我們可以看到，在7/24、7/25、8/14時我們的收盤價大於43，因此在隔天的7/25、7/26、8/15交易，同時因為7/26和之後下的單子，因為收盤價一直沒有達到43(看下面closing_price)，從7/26到8/13的單子都累積到了8/14才各自用43.3 * 1000股成交。成交時'stop_reached'也會變成True。

### 每日收盤價

In [15]:
closing_price

,mdate,coid,close_d
None,,,
0,2018-07-24,1101,45.50
1,2018-07-25,1101,45.10
2,2018-07-26,1101,40.50
3,2018-07-27,1101,40.30
4,2018-07-30,1101,40.70
5,2018-07-31,1101,39.35
6,2018-08-01,1101,41.05
7,2018-08-02,1101,40.60
8,2018-08-03,1101,40.45


在下表可以看到，第一個1000，是分割前就成交的(7/24下單25成交)。7/25下單在7/26除息日成交，所以買入1100，原本持有的1000也在那天變成1100，共2200。從7/26到8/13共計13交易日的單子，則全部在8/14成交，因為下單跟成交都在分割後發生，沒有受到影響，每一單都是1000股。 
7/26收盤價40.5 > 43/1.1，所以以40.5 * 1100股成交。

In [16]:
positions

,,,amount,cost_basis,last_sale_price
date,asset,symbol,,,
2018-07-25 00:00:00+08:00,Equity(0 [1101]),1101,1000,45.164268,45.10
2018-07-26 00:00:00+08:00,Equity(0 [1101]),1101,2200,40.803856,40.50
2018-07-27 00:00:00+08:00,Equity(0 [1101]),1101,2200,40.803856,40.30
2018-07-30 00:00:00+08:00,Equity(0 [1101]),1101,2200,40.803856,40.70
2018-07-31 00:00:00+08:00,Equity(0 [1101]),1101,2200,40.803856,39.35
2018-08-01 00:00:00+08:00,Equity(0 [1101]),1101,2200,40.803856,41.05
2018-08-02 00:00:00+08:00,Equity(0 [1101]),1101,2200,40.803856,40.60
2018-08-03 00:00:00+08:00,Equity(0 [1101]),1101,2200,40.803856,40.45
2018-08-06 00:00:00+08:00,Equity(0 [1101]),1101,2200,40.803856,40.35


In [17]:
transactions

amount  \
date                      asset            symbol           
2018-07-25 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101      1100   
2018-08-14 00:00:00+08:00 Equity(0 [1101]) 1101      1000   
                                           1101      1000   
                                           1101      1000   
                                           1101      1000   
                                           1101      1000   
                                           1101      1000   
                                           1101      1000   
                                           1101      1000   
                                           1101      1000   
                                           1101      1000   
                                           1101      1000   
                                           1101      1000   
                                           1101      1000   

                                                                         dt  \
date                      asset            symbol                             
2018-07-25 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-25 13:30:00+08:00   
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101   2018-07-26 13:30:00+08:00   
2018-08-14 00:00:00+08:00 Equity(0 [1101]) 1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   
                                           1101   2018-08-14 13:30:00+08:00   

                                                   price  \
date                      asset            symbol          
2018-07-25 00:00:00+08:00 Equity(0 [1101]) 1101     45.1   
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101     40.5   
2018-08-14 00:00:00+08:00 Equity(0 [1101]) 1101     43.3   
                                           1101     43.3   
                                           1101     43.3   
                                           1101     43.3   
                                           1101     43.3   
                                           1101     43.3   
                                           1101     43.3   
                                           1101     43.3   
                                           1101     43.3   
                                           1101     43.3   
                                           1101     43.3   
                                           1101     43.3   
                                           1101     43.3   

                                                                           order_id  \
date                      asset            symbol                                     
2018-07-25 00:00:00+08:00 Equity(0 [1101]) 1101    68e00297ce2e41ea91b4298e1c10870f   
2018-07-26 00:00:00+08:00 Equity(0 [1101]) 1101    e5132d7056a3466296723476d5976089   
2018-08-14 00:00:00+08:00 Equity(0 [1101]) 1101    b3587b634b964921a23fe57a6495ed42   
                                           1101    52ac7c7ee3fa4f18b63e58c5125f23eb   
                                           1101    2fd67c9b6a764227b24fb600129d8a0c   
                                       

<span id="order_target"></span>
# Zipline order_target
[Return to Menu](#menu)

order_target的參數和order一樣，差別是他會買/賣讓手上股票數量達到指定的'amount'，而不像order直接買/賣該'amount'。

## 把本單元所有概念結合
### 設定交易策略
#### handle_data

    1. if context.i == 0:
        在模擬的第一個時間點（context.i 為 0），對投資組合中的每個資產下單購買 1000 單位。
        
    2. if context.i == 1:
        在模擬的第二個時間點（context.i 為 1），對投資組合中的每個資產進行調整，使其持有量達到 1100 單位。    
        
    3. if context.i == 3:
        在模擬的第四個時間點（context.i 為 3），對投資組合中的每個資產進行調整，使其持有量達到 2000 單位。
    
    4. if context.i == 5:
        在模擬的第六個時間點（context.i 為 5），對投資組合中的每個資產進行調整，使其持有量達到 3000 單位。此外，此訂單還設定了（stop_price）為 40 和（limit_price）為 40.3，代表當股價在 40 到 40.3 之間時進行交易。
        
    5. record(close=data.current(context.asset, 'close'))
        context.i += 1
        記錄每個資產的收盤價，並將 context.i 遞增 1，表示模擬進入下一個時間點。

In [18]:
def initialize(context):
    context.i = 0
    context.tickers = ['1101']
    context.asset = [symbol(ticker) for ticker in context.tickers]
    set_slippage(slippage.FixedSlippage(spread=0.00))
    set_commission(commission.PerDollar(cost=commission_cost))
    set_benchmark(symbol('IR0001'))

def handle_data(context, data):

    if context.i == 0:
        for asset in context.asset:
            order(asset, 1000)

    if context.i == 1:
        for asset in context.asset:
            order_target(asset, 1100)

    if context.i == 3:
        for asset in context.asset:
            order_target(asset, 2000)

    if context.i == 5:
        for asset in context.asset:
            order_target(asset, 3000, stop_price = 40, limit_price = 40.3)

    record(close=data.current(context.asset, 'close'))
    context.i += 1

def analyze(context, perf):

    pass

commission_cost = 0.001425
capital_base = 1e6
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',
                                      term = '1y',
                                      symbol = '5844')



In [20]:
closing_price = tejapi.get('TWN/APIPRCD',coid=['1101'], opts={'columns':['mdate','coid','close_d']}, mdate={'gte':start_dt,'lte':end_dt }, paginate=True)

performance = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

positions, transactions, orders = get_transaction_detail(performance)

In [21]:
closing_price

,mdate,coid,close_d
None,,,
0,2018-07-24,1101,45.50
1,2018-07-25,1101,45.10
2,2018-07-26,1101,40.50
3,2018-07-27,1101,40.30
4,2018-07-30,1101,40.70
5,2018-07-31,1101,39.35
6,2018-08-01,1101,41.05
7,2018-08-02,1101,40.60
8,2018-08-03,1101,40.45


## 講解

## 1

7/24時用最基本的order功能買1000股的台泥 (1101) ，7/25用order_target將目標股數設為1100，於是程式下單100股 (圖一第二個單)，但在下一個交易日 (7/26) 遇到split， ratio = 1/1.1，所以amount調整成100 / (1/1.1) = 110成交 (圖二)。但同時原本手上的1000股也在split調整成1100，所以7/26收盤時手上共有 1100 + 110 = 1210 股 (圖三)。

In [22]:
performance['orders'][1]

[{'id': '9dbbb7e531c04bb7be455e83857e47e3',
  'dt': Timestamp('2018-07-25 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-07-24 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 1000,
  'commission': 64.2675,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.FILLED: 1>},
 {'id': '1de7225a49974c8390774ca493cbcbb6',
  'dt': Timestamp('2018-07-25 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-07-25 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 100,
  'filled': 0,
  'commission': 0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.OPEN: 0>}]

In [23]:
performance['orders'][2]

[{'id': '1de7225a49974c8390774ca493cbcbb6',
  'dt': Timestamp('2018-07-26 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-07-25 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 110,
  'filled': 110,
  'commission': 6.348375,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.FILLED: 1>}]

In [24]:
positions[0:2]

,,,amount,cost_basis,last_sale_price
date,asset,symbol,,,
2018-07-25 00:00:00+08:00,Equity(0 [1101]),1101,1000,45.164268,45.1
2018-07-26 00:00:00+08:00,Equity(0 [1101]),1101,1210,41.005247,40.5


## 2

在7/27時用order_target將手上股數調整成2000，算出還需要 2000 - 1210 = 790股，下單後隔日成交。

In [25]:
performance['orders'][4]

[{'id': '7426cecdf18444948bb91b32bdaa3243',
  'dt': Timestamp('2018-07-30 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-07-27 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 790,
  'filled': 790,
  'commission': 45.818025000000006,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.FILLED: 1>}]

In [26]:
positions[2:4]

,,,amount,cost_basis,last_sale_price
date,asset,symbol,,,
2018-07-27 00:00:00+08:00,Equity(0 [1101]),1101,1210,41.005247,40.3
2018-07-30 00:00:00+08:00,Equity(0 [1101]),1101,2000,40.907583,40.7


## 3

在7/31時下order_target(asset, 3000, stop_price = 40, limit_price = 40.3)，意思是，當股價超過40後，買1000股將手上2000股調整成3000股，但是限制買入價不能超過40.3。在8/1時股價就超過40，但是一直到8/7時股價低於40.3時才買入1000股。

In [27]:
performance['orders'][5]

[{'id': 'f526d5cdeebb448ba3247971c3e2c835',
  'dt': Timestamp('2018-07-31 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-07-31 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 0,
  'commission': 0,
  'stop': 40.0,
  'limit': 40.300000000000004,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.OPEN: 0>}]

In [28]:
closing_price[4:12]

,mdate,coid,close_d
None,,,
4,2018-07-30,1101,40.70
5,2018-07-31,1101,39.35
6,2018-08-01,1101,41.05
7,2018-08-02,1101,40.60
8,2018-08-03,1101,40.45
9,2018-08-06,1101,40.35
10,2018-08-07,1101,40.15
11,2018-08-08,1101,40.60


In [29]:
performance['orders'][10]

[{'id': 'f526d5cdeebb448ba3247971c3e2c835',
  'dt': Timestamp('2018-08-07 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2018-07-31 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 1000,
  'commission': 57.21375,
  'stop': None,
  'limit': 40.300000000000004,
  'stop_reached': False,
  'limit_reached': True,
  'sid': Equity(0 [1101]),
  'status': <ORDER_STATUS.FILLED: 1>}]

In [30]:
positions[4:10]

,,,amount,cost_basis,last_sale_price
date,asset,symbol,,,
2018-07-31 00:00:00+08:00,Equity(0 [1101]),1101,2000,40.907583,39.35
2018-08-01 00:00:00+08:00,Equity(0 [1101]),1101,2000,40.907583,41.05
2018-08-02 00:00:00+08:00,Equity(0 [1101]),1101,2000,40.907583,40.60
2018-08-03 00:00:00+08:00,Equity(0 [1101]),1101,2000,40.907583,40.45
2018-08-06 00:00:00+08:00,Equity(0 [1101]),1101,2000,40.907583,40.35
2018-08-07 00:00:00+08:00,Equity(0 [1101]),1101,3000,40.674127,40.15
